## Create Object store continairs (S3 buckets) 
we will use S3-compatible buckets to store our:

  - `fancyproject-data`: for datasets
  - `fancyproject-mlflow-metrics`: for metrics, logs
  - `fancyproject-mlflow-artifacts`: for model checkpoints, images, etc.

> These buckets will be used in later notebooks for data storage and experiment tracking and will be mounted to your localfilesystem when configuring your server via a notebook.

The notebook cells uses [swiftclient](https://docs.openstack.org/python-swiftclient/latest/) + [python-chi](https://python-chi.readthedocs.io/en/latest/index.html) 
to automate object store bucket creation on Chameleon

###  Prerequisites
- This notebook assumes that You must be logged into Chameleon JupyterHub and you run the following cells there .

In [ ]:
#Import required packages
import chi
import swiftclient
from chi import context

In [ ]:
project_name = "fancyproject" # Define project name (this will prefix the bucket names)
context.choose_project() 
context.choose_site(default="CHI@TACC")

In [ ]:
# Authenticate - getting object store endpoint
os_conn = chi.clients.connection()
token = os_conn.authorize()
storage_url = os_conn.object_store.get_endpoint()
# Connect to Swift (S3-compatible) object store
swift_conn = swiftclient.Connection(preauthurl=storage_url,
                                    preauthtoken=token,
                                    retries=5)

In [ ]:
# List of buckets to create
buckets = [
    f"{project_name}-data",
    f"{project_name}-mlflow-metrics",
    f"{project_name}-mlflow-artifacts"
]
# Creating buckets
for bucket in buckets:
    print(f"Creating bucket: {bucket}")
    swift_conn.put_container(bucket)

### You can check your buckets in Horizon GUI

Now that your S3 buckets are created, you can verify them in the Chameleon Horizon interface.

1. Log in to [Chameleon Horizon](https://chi.tacc.chameleoncloud.org)
2. In the left-hand menu, go to **Project → Object Store → Containers**.
3. You should see the buckets you just created here.